🎓 **EduDebate AI**
**Multi-Agent Learning Through Structured AI Debate**



“Learn complex topics by watching AI agents reason, argue, and teach.”

🧩 **Overview**
EduDebate AI turns complex, evolving subjects into interactive learning experiences.
It orchestrates several specialized LLM agents using the Google Agent Development Kit (ADK) and Gemini models to stage a structured debate that ends with a concise, student-friendly summary.
Learners don’t just read — they see reasoning unfold through a Pro vs Con debate grounded in recent research.

**Problem Statement:**

Students, analysts, and researchers often struggle to understand complex, evolving topics that require examining multiple perspectives — for example, “Is quantum computing the future of AI?” Traditional search and summarization tools provide facts but not reasoning.


**Solution Overview:**

EduDebate AI transforms learning into an interactive reasoning process by combining several specialized LLM agents under one orchestration layer.
The system hosts an AI-driven debate to explore both sides of a concept, then distills the discussion into structured learning materials.

**Architecture Overview:**

User → Root Agent (EduDebate Coordinator)
        ├── AgentTool(Research Agent + google_search)
        ├── AgentTool(Pro Agent)
        ├── AgentTool(Con Agent)
        └── AgentTool(Scribe-Tutor)
Root Agent (Gemini 2.5-Flash): coordinates workflow and calls each sub-agent as a tool.
Research Agent (Gemini 2.5-Flash): performs Google Search to gather 5-7 authoritative sources.
Pro / Con Agents (Gemini 2.5-Flash): generate structured arguments with citations [src_k].
Scribe-Tutor Agent (Gemini 2.0-Flash): summarizes debate into notes, glossary, and quiz.
calc Tool: lightweight numeric helper for quantitative claims.
InMemoryRunner + LoggingPlugin: provides observability and structured logs across all agents.

**Flow Summary:**

User provides a topic + learning level.
Root Agent calls Research Agent → collects sources.
Root Agent invokes Pro and Con Agents → structured debate.
Root Agent calls Scribe-Tutor → produces educational output.
LoggingPlugin tracks each agent/tool event.

**README / Notebook Includes:**

Step-by-step setup (pip install google-adk >= 1.18.0).
Kaggle Secrets setup for GOOGLE_API_KEY.
Diagram of AgentTool flow.
Example output snippets (sources → debate → learning pack).
Observability log examples from LoggingPlugin.
Reproducibility instructions for both local and Kaggle environments.

**Setup and Initial Auth**

google-adk is already installed in kaggle . So skipping it

In [1]:
pip install google-adk>=1.18.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 which is incompatible.
Note: you may need to restart the kernel to use updated pac

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


Import ADK components

In [3]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents.callback_context import CallbackContext
from google.adk.plugins.base_plugin import BasePlugin

# Built-in Google tool (Gemini 2 compatible)
from google.adk.tools import google_search,AgentTool

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

Python imports

In [5]:
import argparse
import asyncio
import logging
import math
import os
import time
from dataclasses import dataclass
from typing import Any, Dict, Optional

**Simple Calc tool (custom tool for agent)**
Custom tool to do basic math functions
calc Tool: lightweight numeric helper for quantitative claims.

In [6]:
def calc(expression: str) -> Dict[str, Any]:
    """
    Safe(ish) mini math evaluator: supports Python 'math' only.
    Examples: "2*(3+4)", "math.log(8, 2)"
    """
    allowed = {"__builtins__": {}, "math": math}
    try:
        result = eval(expression, allowed, {})
        return {"status": "success", "result": result}
    except Exception as e:
        return {"status": "error", "error": str(e)}

**Models**
we are going to use Gemini models for this

In [7]:
RESEARCH_MODEL  = "gemini-2.5-flash"
DEBATE_MODEL    = "gemini-2.5-flash"
TUTOR_MODEL     = "gemini-2.0-flash"
ROOT_MODEL      = "gemini-2.5-flash"

**Observability (Using Logging Plugin)**
A Plugin in Agent Development Kit (ADK) is a custom code module that can be executed at various stages of an agent workflow lifecycle using callback hooks. You use Plugins for functionality that is applicable across your agent workflow.
An ADK Plugin extends the BasePlugin class and contains one or more callback methods, indicating where in the agent lifecycle the Plugin should be executed. You integrate Plugins into an agent by registering them in your agent's Runner class.
Logging: Log important information at each agent workflow callback point.

In [8]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
) 


**Agents (5): Research,Pro, Con, Scribe-Tutor, Moderator(root)**
* Root Agent (Gemini 2.5-Flash): coordinates workflow and calls each sub-agent as a tool.
* Research Agent (Gemini 2.5-Flash): performs Google Search to gather 5-7 authoritative sources.
* Pro / Con Agents (Gemini 2.5-Flash): generate structured arguments with citations [src_k].
* Scribe-Tutor Agent (Gemini 2.0-Flash): summarizes debate into notes, glossary, and quiz.

**Sessions & State (InMemoryRunner)**
Maintains context automatically per run


In [9]:
COMMON_SUFFIX = (
    "Use the google_search tool to find recent or authoritative sources when needed.\n"
    "Include citation tags like [src_k] (e.g., [src_1]) near each claim.\n"
    "Prefer primary sources, official reports, or reputable orgs. Avoid fallacies; quantify when helpful.\n"
)
research_agent = LlmAgent(
    name="research_agent",
    model=Gemini(model=RESEARCH_MODEL,retry_options=retry_config),
    description="Finds and condenses authoritative recent sources for a topic.",
    instruction=(
        "Use google_search to find 5–7 recent, authoritative sources on the topic. "
        "Prefer primary/official publications (.gov/.edu/.org), reputable journals, or major technical blogs. "
        "Return a numbered list [src_1]..[src_k] with:\n"
        "• Title\n• URL\n• 1–2 sentence takeaway (why it matters)\n"
        "Avoid duplicates; ensure recency where relevant."
    ),
    tools=[google_search],
)
# 2) Pro agent — argues FOR (no web tools needed, cites [src_k] from research)
pro_agent = LlmAgent(
    name="pro_agent",
    model=Gemini(model=DEBATE_MODEL,retry_options=retry_config),
    description="Argues FOR the claim with structured reasoning and citations.",
    instruction=(
        "Produce EXACTLY 3 numbered claims FOR the claim.\n"
        "For each claim: 1–2 sentence justification. Cite ONLY [src_k] provided by the research step.\n"
        "Quantify when helpful; avoid fallacies."
    ),
    tools=[calc],
)

# 3) Con agent — argues AGAINST
con_agent = LlmAgent(
    name="con_agent",
    model=Gemini(model=DEBATE_MODEL,retry_options=retry_config),
    description="Argues AGAINST the claim with structured reasoning and citations.",
    instruction=(
        "Produce EXACTLY 3 numbered counter-claims AGAINST the claim.\n"
        "For each claim: 1–2 sentence justification. Cite ONLY [src_k] provided by the research step.\n"
        "Quantify when helpful; avoid fallacies."
    ),
    tools=[calc],
)

# 4) Scribe-Tutor — converts debate into learning pack
scribe_tutor = LlmAgent(
    name="scribe_tutor",
    model=Gemini(model=TUTOR_MODEL,retry_options=retry_config),
    description="Converts debate transcript and sources into student-ready materials.",
    instruction=(
        "Create a concise learning pack based on the debate & sources:\n"
        "1) Pros/Cons table (Point | Impact | Evidence strength)\n"
        "2) 5 key takeaways\n"
        "3) 5-term glossary (term: short definition)\n"
        "4) 5-question quiz (MCQ or short answer) with 1-sentence rationale each.\n"
        "Keep the reading level aligned to the user's level if provided."
    ),
)

# =============================================================================
# Root agent — orchestrates via AgentTool(...) calls
# =============================================================================

root_agent = LlmAgent(
    name="edudebate_root",
    model=Gemini(model=ROOT_MODEL,retry_options=retry_config),
    description="Root coordinator for research→debate→tutor educational flow.",
    instruction=(
        "You orchestrate an educational debate using callable tools:\n"
        "1) Call Research Agent to get 5–7 sources on the given topic. Keep the returned list as [src_k].\n"
        "2) Call Pro Agent to produce exactly 3 FOR claims that cite [src_k].\n"
        "3) Call Con Agent to produce exactly 3 AGAINST claims that cite [src_k].\n"
        "4) Ask each side for a brief 120–150 word closer summarizing their strongest points with [src_k].\n"
        "5) Call Scribe-Tutor to generate the learning pack (notes, pros/cons, glossary, quiz).\n"
        "Return a clean final answer with: the sources, both sides' closers, and the learning pack."
    ),
    tools=[
        AgentTool(research_agent),
        AgentTool(pro_agent),
        AgentTool(con_agent),
        AgentTool(scribe_tutor),
        calc,  # optional numeric helper for root if needed
    ],
)
# Runner with standardized observability
runner = InMemoryRunner(agent=root_agent, plugins=[LoggingPlugin()])


Orchestration (With Runner and InMemorySessionService)

In [10]:
async def main(topic: str, level: str = "beginner"):
    """
    One-shot debug run. The root agent will call Research→Pro→Con→Tutor via AgentTool.
    """
    prompt = f"Topic: {topic}\nLevel: {level}\n\nPlease follow the orchestration steps."
    resp = await runner.run_debug(prompt)

    # ADK versions may return a single object or list; try best-effort printing
    try:
        print("\n🎓 EduDebate AI — OUTPUT\n" + "="*80)
        print(getattr(resp, "output_text", "") or str(resp))
    except Exception:
        # Fallback: if it's a list, print each item
        if isinstance(resp, list):
            print("\n🎓 EduDebate AI — OUTPUT (list)\n" + "="*80)
            for i, r in enumerate(resp, 1):
                print(f"\n--- PART {i} ---\n")
                print(getattr(r, "output_text", "") or str(r))
        else:
            print(resp)

# Optional script entry (if run as a .py outside notebooks)
if __name__ == "__main__":
    import asyncio
    try:
        asyncio.get_running_loop()
        print("⚠️ Detected running event loop. In notebooks, call: await main(...).")
    except RuntimeError:
        asyncio.run(main("What are the latest advancements in quantum computing and what do they mean for AI?", "intermediate"))

⚠️ Detected running event loop. In notebooks, call: await main(...).


In [11]:
# In a Kaggle/Jupyter cell after the file is loaded/run:
await main(
    "What are the latest advancements in quantum computing and what do they mean for AI?",
    level="intermediate"
)


 ### Created new session: debug_session_id

User > Topic: What are the latest advancements in quantum computing and what do they mean for AI?
Level: intermediate

Please follow the orchestration steps.
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-316a5a1c-a4cd-478a-8120-0df07f900ff2
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: edudebate_root
[logging_plugin]    User Content: text: 'Topic: What are the latest advancements in quantum computing and what do they mean for AI?
Level: intermediate

Please follow the orchestration steps.'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-316a5a1c-a4cd-478a-8120-0df07f900ff2
[logging_plugin]    Starting Agent: edudebate_root
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: edudebate_root
[logging_plugin]    Invocation ID: e-316a5a1c-a4cd-478a-8120

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: research_agent
[logging_plugin]    Token Usage - Input: 466, Output: 24
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 751ef846-bbaa-4dcd-a7a2-c470a0ad89c4
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: research_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['research_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: research_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-6436e88f-d7f5-4e85-b4a8-5bd565e85a18
[logging_plugin]    Arguments: {'request': 'latest advancements in quantum computing and their implications for AI'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-be7f8b74-6a10-4bd5-b19d-917144369f3f
[logging_plugin]    Session ID: db9f7d27-c9fd-4910-ae5e-c8e992ee9512
[logging_p

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: pro_agent
[logging_plugin]    Token Usage - Input: 3503, Output: 48
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: e6a3e373-630b-4499-89cd-62db9cdc198d
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: pro_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['pro_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: pro_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-c9cdd1fd-ba19-4864-823d-3a00d1f6fde0
[logging_plugin]    Arguments: {'request': 'Based on the provided sources, present 3 FOR claims regarding the latest advancements in quantum computing and what they mean for AI. Cite each claim using [src_k]'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-b5292a6d-5cdb-4791-b11f-727dab687e92
[logg

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: pro_agent
[logging_plugin]    Content: text: 'I cannot produce claims as no research has been provided. Please provide me with the sources (src_k).'
[logging_plugin]    Token Usage - Input: 184, Output: 22
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 763d9a9f-d4c5-4110-aa4f-0b553bf1e59c
[logging_plugin]    Author: pro_agent
[logging_plugin]    Content: text: 'I cannot produce claims as no research has been provided. Please provide me with the sources (src_k).'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: pro_agent
[logging_plugin]    Invocation ID: e-b5292a6d-5cdb-4791-b11f-727dab687e92
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    Invocation ID: e-b5292a6d-5cdb-4791-b11f-727dab687e92
[logging_plugin]    Final Agent: pro_agent
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: pro_agent
[logging_plugin]    Agent: edudeba

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: pro_agent
[logging_plugin]    Token Usage - Input: 3587, Output: 3025
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 60da6959-68a3-4f43-965d-576ec5f2b242
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: pro_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['pro_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: pro_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-0e3e6d98-aeeb-4b73-999c-61c7b78a1ebb
[logging_plugin]    Arguments: {'request': 'Based on the following sources, present 3 FOR claims regarding the latest advancements in quantum computing and what they mean for AI. Cite each claim using [src_k].\n\nSources:\n*   **[src_1] A new quantum toolkit for optimization - Google Research**\n    *   **URL**: [https://verte

...

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: pro_agent | function_call: con_agent
[logging_plugin]    Token Usage - Input: 10575, Output: 1369
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 56766a86-43a5-425e-86d8-2bd844748507
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: pro_agent | function_call: con_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['pro_agent', 'con_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: pro_agent
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-2833fe2a-5bca-4db9-acb3-61219972ab64
[logging_plugin]    Arguments: {'request': 'Provide a 120-150 word closer summarizing the strongest FOR points for the topic: \'What are the latest advancements in quantum computing and what do they mean for AI?\', citing sources with [src_k].\n\n**Pro Claims:

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: con_agent
[logging_plugin]    Content: text: 'Despite significant discourse, the immediate and widespread impact of quantum computing on AI remains largely prospective rather than present. Current advancements underscore that quantum computing fo...'
[logging_plugin]    Token Usage - Input: 816, Output: 255
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: de9b4dc7-507c-4c72-bba4-52b7eec4fa01
[logging_plugin]    Author: con_agent
[logging_plugin]    Content: text: 'Despite significant discourse, the immediate and widespread impact of quantum computing on AI remains largely prospective rather than present. Current advancements underscore that quantum computing fo...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: con_agent
[logging_plugin]    Invocation ID: e-87523c47-6c33-4571-bf54-ad990fab8a3a
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    I

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: pro_agent
[logging_plugin]    Content: text: 'Quantum computing advancements are profoundly enhancing AI capabilities, starting with enhanced optimization. Google Quantum AI’s Decoded Quantum Interferometry (DQI) algorithm, for instance, provides...'
[logging_plugin]    Token Usage - Input: 799, Output: 167
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 83b8fc0c-ac89-4c1e-95ec-ed80acb3aabc
[logging_plugin]    Author: pro_agent
[logging_plugin]    Content: text: 'Quantum computing advancements are profoundly enhancing AI capabilities, starting with enhanced optimization. Google Quantum AI’s Decoded Quantum Interferometry (DQI) algorithm, for instance, provides...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: pro_agent
[logging_plugin]    Invocation ID: e-099b7ada-bcb9-474d-a072-b5ee6d67cfe8
[logging_plugin] ✅ INVOCATION COMPLETED
[logging_plugin]    I

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: function_call: scribe_tutor
[logging_plugin]    Token Usage - Input: 12395, Output: 3913
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: b00a50e9-e90c-4755-896a-895ee85724f8
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: function_call: scribe_tutor
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['scribe_tutor']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: scribe_tutor
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Function Call ID: adk-fc042b3a-688f-4a86-985f-2a61c0d93d57
[logging_plugin]    Arguments: {'request': 'Topic: What are the latest advancements in quantum computing and what do they mean for AI?\nLevel: intermediate\n\nSources:\n*   **[src_1] A new quantum toolkit for optimization - Google Research**\n    *   **URL**: [https://vertexaisearch.cloud.google.com/grounding-api-

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: edudebate_root
[logging_plugin]    Content: text: '## Debate Topic: What are the latest advancements in quantum computing and what do they mean for AI?
### Level: Intermediate

### Sources:

*   **[src_1] A new quantum toolkit for optimization - Googl...'
[logging_plugin]    Token Usage - Input: 17573, Output: 4477
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: b53f7254-19ea-4bd7-8f55-090489194aa1
[logging_plugin]    Author: edudebate_root
[logging_plugin]    Content: text: '## Debate Topic: What are the latest advancements in quantum computing and what do they mean for AI?
### Level: Intermediate

### Sources:

*   **[src_1] A new quantum toolkit for optimization - Googl...'
[logging_plugin]    Final Response: True
edudebate_root > ## Debate Topic: What are the latest advancements in quantum computing and what do they mean for AI?
### Level: Intermediate

### Sources:

*   **[src_1] A new quantum toolkit for op